In [19]:
#Load packages
import numpy as np
import pandas as pd
import random
import torch
import json
from torch import nn
import torch.nn.functional as F
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

device = torch.device("cpu")

In [2]:
with open('labtoinfo.json') as f:
    labtoinfo = json.load(f)
with open('codetolab.json') as f:
    codetolab = json.load(f)

In [4]:
with open('conv.txt') as f:
    lines = f.readlines()
conv = []
for i in lines:
    conv.append(i.strip())

In [5]:
convlab = []
for i in conv:
    convlab.append(str(codetolab[i]))

In [6]:
labpool = []
for i in labtoinfo:
    if i not in convlab:
        labpool.append(i)

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
indexer = nlp.data.BERTSentenceTransform(tok, max_seq_length=32, pad=True, pair=False)

using cached model
using cached model
using cached model


In [8]:
class BERTClassification(nn.Module):
    def __init__(self, bert, hidden_size = 768, dr_rate = .5):
        super(BERTClassification, self).__init__()
        self.bert = bert
        self.d = nn.Dropout(p = dr_rate)
        self.linear = nn.Linear(hidden_size, 531)
        
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        out = self.d(pooler)
        out = self.linear(out)
        return out

In [9]:
model = BERTClassification(bertmodel).to(device)
model.load_state_dict(torch.load('COVID_Class_BERT4.pth',map_location=device))
model.eval()

BERTClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [10]:
def covidbertwans(inp):
    token_ids, valid_length, segment_ids = indexer([inp])
    token_ids = torch.tensor(token_ids).view(1,len(token_ids)).long().to(device)
    segment_ids = torch.tensor(segment_ids).long().to(device)
    valid_length= torch.tensor(valid_length).view(-1)
    result = model(token_ids, valid_length, segment_ids).view(-1).detach().numpy()
    pred = np.argmax(result)
    top10pred = np.argsort(result)[::-1][:10]
    score = F.softmax(torch.tensor(result),dim=0).detach().numpy()\
    [np.argsort(F.softmax(torch.tensor(result),dim=0)).detach().numpy()[::-1][:10]]
    
    #### 답변을 찾지 못할경우 표시
    if score[0] < .5:
        print('죄송합니다ㅠ 질문에 대한 답변을 찾지 못했어요.')
        print(' ')
        print('혹시 이런게 궁금하신가요?:')
        count = 0
        j=1
        i=1
        while count <= 4:
            if str(top10pred[i]) in convlab:
                i+=1
            else :
                print(j,'. ',labtoinfo[str(top10pred[i])]['ques']) #, ' ', score[i-1]
                i+=1
                j+=1
                count+=1
    else :
        #### 답변 표시
        top1 = top10pred[0]
        
        if str(top1) in convlab:
            #print('질문: ',labtoinfo[str(top1)]['ques'], ' ', score[0])
            print(' ')
            print('답변:')
            print(labtoinfo[str(top1)]['txtans'])
        else:
            print('질문: ',labtoinfo[str(top1)]['ques']) #, ' ', score[0]
            print(' ')
            print('답변:')
            if str(labtoinfo[str(top1)]['txtans'])!='nan':
                print(labtoinfo[str(top1)]['txtans'])
            if str(labtoinfo[str(top1)]['imgans'])!='nan':
                direc = str(labtoinfo[str(top1)]['imgans'])
                img = mpimg.imread(direc,0)
                plt.figure(figsize=(10,10))
                imgplot = plt.imshow(img)
                plt.show()
            if str(labtoinfo[str(top1)]['ref'])!='nan':
                print('출처: ',labtoinfo[str(top1)]['ref'])

        #### 추천질문 표시  
        if str(top1) in convlab:
            j=1
            labs = random.choices(labpool,k=5)
            print(' ')
            print('혹시 이런게 궁금하신가요?:')
            for i in labs:
                print(j,'. ',labtoinfo[i]['ques'])
                j+=1
        else:
            count = 0
            i = 1
            j = 1
            print(' ')
            print('비슷한 질문들도 있어요:')
            while count <= 3:
                if str(top10pred[i]) in convlab:
                    i+=1
                else :
                    print(j,'. ',labtoinfo[str(top10pred[i])]['ques']) #, ' ', score[i-1]
                    i+=1
                    j+=1
                    count+=1

In [15]:
inp = input()
covidbertwans(inp)

코로나 바이러스 증상
질문:  코로나19의 증상은 어떤 것이 있나요?
 
답변:
코로나19의 가장 흔한 증상은 발열, 마른 기침, 피로이며 그 외에 후각 및 미각 소실, 근육통, 인후통, 콧물, 코막힘, 두통, 결막염, 설사, 피부 증상 등 다양한 증상이 나타날 수 있습니다.
이러한 증상은 보통 경미하고 점진적으로 나타납니다. 어떤 사람들은 감염되어도 매우 약한 증상만 나타날 수 있습니다.
대부분의 환자들(약 80%)은 특별한 치료 없이 회복되나, 5명 중 1명 정도는 중증으로 진행할 수 있습니다. 고령자나 고혈압, 심폐질환, 당뇨병이나 암과 같은 기저질환이 있는 사람들은 중증으로 진행될 가능성이 높습니다.
출처:  http://ncov.mohw.go.kr/faqBoardList.do?brdId=3&brdGubun=38
 
비슷한 질문들도 있어요:
1 .  무증상자는 어떻게 전염을 시키나요?
2 .  코로나 19에 걸린 어린이의 증상은 무엇인가요?
3 .  코로나바이러스감염증-19는 어떤 질병인가요?
4 .  코로나-19를 검사하는데 과거 병력은 왜 물어보나요?
